# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Word2Vec
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Word2Vec Model

### Twitter

In [9]:
taggerName = 'ct'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 987049.28it/s] 


(442359, 159113)
vocab size : 159113


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [13]:
modelName = bm.Return_ModelName('word2vec', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

XGBoost
NeuralNetwork_2
NeuralNetwork_1
LogisticRegression
SVC
RandomForestClassifier


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  6%|▌         | 9161/162640 [00:00<00:01, 84855.33it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 118447.04it/s]
1it [00:00,  9.78it/s]

running time : 0:00:01.377179
Vectorizing Data


9372it [01:25, 109.37it/s]


scaling Data
total running time : 0:01:27.529258


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 425767.85it/s]
9372it [00:00, 593173.54it/s]
9372it [00:00, 670905.38it/s]


CPU times: user 2.74 s, sys: 489 ms, total: 3.23 s
Wall time: 3.1 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

  9%|▊         | 13977/162640 [00:00<00:01, 139727.82it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 127683.76it/s]
6it [00:00, 59.41it/s]

running time : 0:00:01.277545
Vectorizing Data


15120it [02:44, 91.74it/s]


scaling Data
total running time : 0:02:47.015474


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 728647.97it/s]
15120it [00:00, 661477.96it/s]
15120it [00:00, 548858.68it/s]


CPU times: user 4.45 s, sys: 629 ms, total: 5.08 s
Wall time: 5.32 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('word2vec', model2,'ct')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  5%|▌         | 8747/162640 [00:00<00:01, 87400.07it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 118045.32it/s]
2it [00:00, 14.00it/s]

running time : 0:00:01.381967
Vectorizing Data


9372it [01:23, 112.44it/s]


scaling Data
total running time : 0:01:25.129047


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 417368.50it/s]
9372it [00:00, 773266.79it/s]
9372it [00:00, 859908.93it/s]


CPU times: user 25.9 s, sys: 1.59 s, total: 27.5 s
Wall time: 26.1 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  6%|▌         | 9387/162640 [00:00<00:01, 93833.40it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 99083.48it/s]
7it [00:00, 65.67it/s]

running time : 0:00:01.645887
Vectorizing Data


15120it [02:21, 107.15it/s]


scaling Data
total running time : 0:02:23.667643


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 828861.83it/s]
15120it [00:00, 720273.00it/s]
15120it [00:00, 854239.37it/s]


CPU times: user 43.5 s, sys: 1.83 s, total: 45.3 s
Wall time: 44.2 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('word2vec', model3,'ct')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  8%|▊         | 13009/162640 [00:00<00:01, 130054.78it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 135558.54it/s]
13it [00:00, 126.72it/s]

running time : 0:00:01.203458
Vectorizing Data


9372it [01:09, 135.12it/s]


scaling Data
total running time : 0:01:10.867181


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 807017.53it/s]
9372it [00:00, 812572.70it/s]
9372it [00:00, 845227.97it/s]


CPU times: user 30.1 s, sys: 984 ms, total: 31.1 s
Wall time: 30.8 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  8%|▊         | 12459/162640 [00:00<00:01, 124535.19it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:02<00:00, 78568.30it/s]
4it [00:00, 33.77it/s]

running time : 0:00:02.074107
Vectorizing Data


15120it [02:22, 105.88it/s]


scaling Data
total running time : 0:02:25.657547


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 655251.66it/s]
15120it [00:00, 623964.47it/s]
15120it [00:00, 809376.37it/s]


CPU times: user 47.7 s, sys: 1.18 s, total: 48.9 s
Wall time: 47.9 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

In [37]:
taggerName = 'mecab'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [38]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 933790.78it/s] 


(442359, 161590)
vocab size : 161590


#### News to tagged Document

In [39]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [40]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [41]:
modelName = bm.Return_ModelName('word2vec', model1,'mecab')

In [42]:
classifierList = glob(classifierPath+'*'+modelName)

In [43]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [44]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

  5%|▍         | 7997/162640 [00:00<00:01, 79941.87it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 99216.75it/s]
1it [00:00,  7.54it/s]

running time : 0:00:01.645575
Vectorizing Data


9372it [01:46, 88.22it/s]


scaling Data
total running time : 0:01:48.337924


In [45]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 272617.69it/s]
9372it [00:00, 684539.86it/s]
9372it [00:00, 783344.63it/s]


CPU times: user 22.8 s, sys: 1.75 s, total: 24.5 s
Wall time: 23.3 s


In [46]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

  8%|▊         | 12645/162640 [00:00<00:01, 126412.15it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 105078.38it/s]
6it [00:00, 57.48it/s]

running time : 0:00:01.552436
Vectorizing Data


15120it [02:54, 86.79it/s]


scaling Data
total running time : 0:02:56.693834


In [47]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 794859.64it/s]
15120it [00:00, 714760.91it/s]
15120it [00:00, 160521.51it/s]


CPU times: user 35.9 s, sys: 1.85 s, total: 37.7 s
Wall time: 36.1 s


In [48]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [49]:
modelName = bm.Return_ModelName('word2vec', model2,'mecab')

In [50]:
classifierList = glob(classifierPath+'*'+modelName)

In [51]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [52]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

  4%|▍         | 7166/162640 [00:00<00:02, 71632.91it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 97225.57it/s]
2it [00:00, 14.08it/s]

running time : 0:00:01.676400
Vectorizing Data


9372it [01:24, 110.62it/s]


scaling Data
total running time : 0:01:26.750448


In [53]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 816725.89it/s]
9372it [00:00, 864409.39it/s]
9372it [00:00, 812052.33it/s]


CPU times: user 24.6 s, sys: 934 ms, total: 25.5 s
Wall time: 24.3 s


In [54]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  8%|▊         | 12855/162640 [00:00<00:01, 128507.53it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 135210.08it/s]
7it [00:00, 62.86it/s]

running time : 0:00:01.206494
Vectorizing Data


15120it [02:39, 94.69it/s]


scaling Data
total running time : 0:02:41.713046


In [55]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 764641.98it/s]
15120it [00:00, 859670.28it/s]
15120it [00:00, 773992.83it/s]


CPU times: user 39.3 s, sys: 1.58 s, total: 40.9 s
Wall time: 39.1 s


In [56]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [57]:
modelName = bm.Return_ModelName('word2vec', model3,'mecab')

In [58]:
classifierList = glob(classifierPath+'*'+modelName)

In [59]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [60]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  4%|▍         | 7022/162640 [00:00<00:02, 70188.44it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 102681.63it/s]
8it [00:00, 76.17it/s]

running time : 0:00:01.606502
Vectorizing Data


9372it [01:26, 108.19it/s]


scaling Data
total running time : 0:01:28.547343


In [61]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 426483.86it/s]
9372it [00:00, 778211.46it/s]
9372it [00:00, 834391.48it/s]


CPU times: user 29.2 s, sys: 786 ms, total: 29.9 s
Wall time: 28.9 s


In [62]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  7%|▋         | 11648/162640 [00:00<00:01, 116447.63it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:01<00:00, 119653.96it/s]
9it [00:00, 87.28it/s]

running time : 0:00:01.362872
Vectorizing Data


15120it [02:38, 95.31it/s]


scaling Data
total running time : 0:02:40.747204


In [63]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 804734.11it/s]
15120it [00:00, 735228.58it/s]
15120it [00:00, 695523.98it/s]


CPU times: user 46.2 s, sys: 735 ms, total: 46.9 s
Wall time: 45.7 s


In [64]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict